In [1]:
import pandas as pd
import numpy as np

In [2]:
ch2 = pd.read_csv("CH2.csv")
ch3 = pd.read_csv("CH3.csv")
ch4 = pd.read_csv("CH4.csv")

ch2_X = ch2.select_dtypes(exclude = ["int64", "object"]).copy()
ch3_X = ch3.select_dtypes(exclude = ["int64", "object"]).copy()
ch4_X = ch4.select_dtypes(exclude = ["int64", "object"]).copy()

ch2_X.drop(["0"], axis = 1, inplace = True)
ch3_X.drop(["0"], axis = 1, inplace = True)
ch4_X.drop(["0"], axis = 1, inplace = True)

Y = ch2["label"]

In [3]:
def clean(X, Y):
    fft = X.select_dtypes("float64").T

    X = X.drop(np.where(np.max(fft) == np.inf)[0])
    Y = Y.drop(np.where(np.max(fft) == np.inf)[0])

    Y = Y[~X.isna().any(axis = 1)]
    X = X[~X.isna().any(axis = 1)]

    X = X.reset_index(drop=True)
    Y = Y.reset_index(drop=True)

    return X, Y


In [4]:
data = pd.concat([ch2_X, ch3_X, ch4_X], axis = 1)

In [5]:
X, Y = clean(data, Y)

## Normalization

In [6]:
norm = np.reshape(np.linalg.norm(X, axis = 1), (-1,1))
X = X / norm

In [7]:
X

,1,2,3,4,5,6,7,8,9,10,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,5000
0,0.000062,0.000085,0.000046,0.000220,0.000061,0.000069,0.000172,0.000133,0.000075,0.000140,...,0.000028,0.000013,0.000028,0.000024,0.000010,0.000041,0.000039,0.000057,0.000006,0.000002
1,0.000098,0.000096,0.000105,0.000142,0.000046,0.000115,0.000133,0.000084,0.000007,0.000080,...,0.000042,0.000024,0.000030,0.000036,0.000032,0.000030,0.000045,0.000040,0.000031,0.000054
2,0.000134,0.000127,0.000147,0.000177,0.000047,0.000023,0.000065,0.000154,0.000103,0.000058,...,0.000043,0.000075,0.000024,0.000012,0.000023,0.000029,0.000060,0.000048,0.000063,0.000091
3,0.000016,0.000111,0.000029,0.000101,0.000062,0.000075,0.000022,0.000196,0.000139,0.000088,...,0.000024,0.000032,0.000016,0.000022,0.000024,0.000012,0.000026,0.000023,0.000040,0.000006
4,0.000100,0.000121,0.000050,0.000080,0.000103,0.000018,0.000027,0.000044,0.000035,0.000153,...,0.000045,0.000049,0.000045,0.000038,0.000035,0.000041,0.000013,0.000054,0.000021,0.000079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,0.000041,0.000086,0.000066,0.000053,0.000057,0.000071,0.000086,0.000116,0.000206,0.001093,...,0.000025,0.000037,0.000077,0.000006,0.000069,0.000037,0.000021,0.000065,0.000021,0.000007
2600,0.000066,0.000130,0.000114,0.000053,0.000074,0.000132,0.000182,0.000089,0.000063,0.001823,...,0.000103,0.000051,0.000032,0.000042,0.000073,0.000035,0.000061,0.000046,0.000028,0.000105
2601,0.000165,0.000141,0.000020,0.000047,0.000091,0.000167,0.000031,0.000067,0.000102,0.000776,...,0.000018,0.000019,0.000083,0.000073,0.000041,0.000027,0.000084,0.000105,0.000018,0.000089
2602,0.000027,0.000106,0.000044,0.000085,0.000109,0.000052,0.000028,0.000137,0.000158,0.002249,...,0.000016,0.000046,0.000019,0.000048,0.000072,0.000061,0.000025,0.000039,0.000068,0.000129


### Splitting

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0, stratify=Y)

In [11]:
import tensorflow as tf
from keras import Sequential, Input
from keras.layers import Dense
from keras.losses import SparseCategoricalCrossentropy


model = Sequential([Input(shape = (15000,)),
                    Dense(1000, activation = "relu"),
                    Dense(200, activation = "relu"),
                    Dense(50, activation = "relu"),
                    Dense(10, activation = "relu"),
                    Dense(7, activation = "softmax")])

model.compile(loss = SparseCategoricalCrossentropy(), optimizer = "rmsprop", metrics = ["accuracy"])

In [15]:
model.fit(x_train, y_train, epochs = 200)

Epoch 1/50
66/66 [==============================] - 1s 9ms/step - loss: 0.3153 - accuracy: 0.8905
Epoch 2/50
66/66 [==============================] - 1s 8ms/step - loss: 0.3044 - accuracy: 0.8833
Epoch 3/50
66/66 [==============================] - 1s 8ms/step - loss: 0.3638 - accuracy: 0.8944
Epoch 4/50
66/66 [==============================] - 1s 8ms/step - loss: 0.3001 - accuracy: 0.9006
Epoch 5/50
66/66 [==============================] - 1s 8ms/step - loss: 0.3206 - accuracy: 0.8905
Epoch 6/50
66/66 [==============================] - 1s 8ms/step - loss: 0.3218 - accuracy: 0.8809
Epoch 7/50
66/66 [==============================] - 1s 8ms/step - loss: 0.3260 - accuracy: 0.8752
Epoch 8/50
66/66 [==============================] - 1s 8ms/step - loss: 0.2986 - accuracy: 0.8997
Epoch 9/50
66/66 [==============================] - 1s 8ms/step - loss: 0.2690 - accuracy: 0.9054
Epoch 10/50
66/66 [==============================] - 1s 8ms/step - loss: 0.2862 - accuracy: 0.9073
Epoch 11/50
66/66 [